# Example notebook

In [ ]:
include("../impute.jl")
include("../regress.jl")

Read in a data file.

In [ ]:
data = CSV.read("../datasets/sleep-0.20-01.csv")
first(data, 5)

Impute missing data and regress.

In [ ]:
imputed = mice(data);
first(imputed, 5)
linear = regress(imputed)

In [ ]:
R2, OSR2 = evaluate(imputed, linear)
@show R2
@show OSR2

Add indicator variables for missingness and impute zeros (finitely adaptive).

In [ ]:
missingindicators = indicatemissing(data)
augmented = [zeroimpute(data) missingindicators]
linear2 = regress(augmented)

In [ ]:
R2, OSR2 = evaluate(augmented, linear2)
@show R2
@show OSR2